This notebook evaluates and populates a new SGCN-specific set of information in the TIR with a logical taxonomic group value to use in the SGCN National List and elsewhere. It uses a table of logical mappings from the SGCN schema (sgcn.adjustments) from submitted taxonomic groups to those that we want to use for consistency and clarity in the National List. We will update this process in future to build the mapping based on taxonomy from the taxonomic authorities (e.g., "aves" to "birds"), but the data structures in the TIR and SGCN will remain consistent.

In [16]:
import requests,configparser
from IPython.display import display


In [17]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))

In [18]:
# Build base URL with API key using input from the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [29]:
# Retrieve the logical mappings from the sgcn.adjustments table and set them up as a dictionary for ease of use

tgMappings = requests.get(getBaseURL()+"&q=SELECT taxonomicgroup FROM sgcn.adjustments").json()
tgMapper = {}

for feature in tgMappings["features"]:
    key = feature["properties"]["taxonomicgroup"].split("=>")[0].replace('"','')
    value = feature["properties"]["taxonomicgroup"].split("=>")[1].replace('"','')
    tgMapper[key] = value

display (tgMapper)

{'Amphibian': 'Amphibians',
 'Arthropod': 'Other Invertebrates',
 'Bird': 'Birds',
 'Bivalves': 'Mollusks',
 'Bryophytes': 'Plants',
 'Cephalopods': 'Mollusks',
 'Cnidarians': 'Other Invertebrates',
 'Echinoderms': 'Other Invertebrates',
 'Flatworm': 'Other Invertebrates',
 'Freshwater Mussel': 'Mollusks',
 'Gastropods': 'Mollusks',
 'Insect': 'Insects',
 'Invertebrate': 'Other Invertebrates',
 'Invertebrates': 'Other Invertebrates',
 'Mammal': 'Mammals',
 'Mite': 'Arachnids',
 'Mussel': 'Mollusks',
 'Myriapods': 'Other Invertebrates',
 'Plant': 'Plants',
 'Poriferans': 'Other Invertebrates',
 'Reptile': 'Reptiles',
 'Snail': 'Mollusks',
 'Spider': 'Arachnids',
 'Vascular Plants': 'Plants',
 'Worms': 'Other Invertebrates'}

In [42]:
# Retrieve the submitted taxonomic groups from the registration information for SGCN taxa
#r = requests.get(getBaseURL()+"&q=SELECT DISTINCT registration->'taxonomicgroup_submitted' AS taxonomicgroup_submitted FROM tir.tir2 WHERE exist(registration, 'taxonomicgroup_submitted')").json()
r = requests.get(getBaseURL()+"&q=SELECT DISTINCT taxonomicgroup_submitted FROM sgcn.sgcn WHERE taxonomicgroup_submitted <> ''").json()

for feature in r["features"]:
    submittedTG = feature["properties"]["taxonomicgroup_submitted"]
    
    # If possible, map the submitted taxonomic group to one of our adjusted taxonomic groups
    try:
        acceptedTG = tgMapper[submittedTG]
    except:
        acceptedTG = submittedTG

    # Update the TIR with accepted taxonomic groups for all records based on the mapping to submitted taxonomic groups
    sgcnPairs = '"taxonomicgroup"=>"'+acceptedTG+'"'
    q = "UPDATE tir.tir2 SET sgcn = '"+sgcnPairs+"' WHERE registration->'taxonomicgroup_submitted' = '"+submittedTG+"'"
    print (q)
    print (requests.get(getBaseURL()+"&q="+q).json())
    print ("----")
    
    

UPDATE tir.tir2 SET sgcn = '"taxonomicgroup"=>"Other Invertebrates"' WHERE registration->'taxonomicgroup_submitted' = 'Other Invertebrates'
{'affected_rows': 560, '_execution_time': 0.123, 'auth_check': {'session': None, 'success': True, 'checked_relations': ['tir.tir2'], 'auth_level': 'Write'}, 'success': True}
----
UPDATE tir.tir2 SET sgcn = '"taxonomicgroup"=>"Other Invertebrates"' WHERE registration->'taxonomicgroup_submitted' = 'Arthropod'
{'affected_rows': 0, '_execution_time': 0.092, 'auth_check': {'session': None, 'success': True, 'checked_relations': ['tir.tir2'], 'auth_level': 'Write'}, 'success': True}
----
UPDATE tir.tir2 SET sgcn = '"taxonomicgroup"=>"Plants"' WHERE registration->'taxonomicgroup_submitted' = 'Plants'
{'affected_rows': 2603, '_execution_time': 0.281, 'auth_check': {'session': None, 'success': True, 'checked_relations': ['tir.tir2'], 'auth_level': 'Write'}, 'success': True}
----
UPDATE tir.tir2 SET sgcn = '"taxonomicgroup"=>"Birds"' WHERE registration->'taxo